In [ ]:
option files / configuration files 
===========================================

-- are the files where the mysql program reads and sets the option specified in these files while startup.
-- example of mysql programs are mysqld , mysql , mysqladmin , mysqlimport , etc.
-- default location is /etc/my.cnf , can also reside in /etc/mysql/my.cnf /usr/etc/my.cnf ~/.my.cnf.
-- these file reading can be disabled by using --no-defaults.
-- # are used inside these files to indicate comments.
-- option = value is used to set the option and both side spaces are allowed.
-- value can be without quote , single quote and double quote.
-- for option both option or variable can be used , eg : server-id / server_id .
-- there are option groups [mysqld] , [client] , [mysql] , [server] , [mysqladmin]


[root@mysqlserver ~]# mysql --help | grep -B 1 my.cnf

Default options are read from the following files in the given order:
/etc/my.cnf /etc/mysql/my.cnf /usr/etc/my.cnf ~/.my.cnf 


[root@mysqlserver ~]# mysqladmin --help | grep -B 1 my.cnf

Default options are read from the following files in the given order:
/etc/my.cnf /etc/mysql/my.cnf /usr/etc/my.cnf ~/.my.cnf 

[root@mysqlserver ~]# ls -rtl /etc/my.cnf
-rw-r--r--. 1 root root 1243 May 16 01:37 /etc/my.cnf
[root@mysqlserver ~]# 

[root@mysqlserver ~]# ls -rtl /etc/mysql/my.cnf
ls: cannot access '/etc/mysql/my.cnf': No such file or directory
[root@mysqlserver ~]# 

[root@mysqlserver ~]# ls -rtl /usr/etc/my.cnf
ls: cannot access '/usr/etc/my.cnf': No such file or directory
[root@mysqlserver ~]# 

In [ ]:
Remove comments from option file , add an option , restart and verify
=================================================================================

[root@mysqlserver ~]# cp -p /etc/my.cnf /etc/my.cnf.bkp
[root@mysqlserver ~]# 
[root@mysqlserver ~]# vi /etc/my.cnf
[root@mysqlserver ~]# 
[root@mysqlserver ~]# cat /etc/my.cnf
[mysqld]

datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock

log-error=/var/log/mysqld.log
pid-file=/var/run/mysqld/mysqld.pid

server_id = 10
[root@mysqlserver ~]# 

[root@mysqlserver ~]# systemctl restart mysqld
[root@mysqlserver ~]# 

mysql> show variables like 'server%';
+----------------+--------------------------------------+
| Variable_name  | Value                                |
+----------------+--------------------------------------+
| server_id      | 10                                   |
| server_id_bits | 32                                   |
| server_uuid    | 486f86fb-11db-11ef-9f50-000c29a50cb9 |
+----------------+--------------------------------------+
3 rows in set (0.02 sec)


[root@mysqlserver ~]# vi /etc/my.cnf
[root@mysqlserver ~]# 
[root@mysqlserver ~]# cat /etc/my.cnf
[mysqld]

datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock

log-error=/var/log/mysqld.log
pid-file=/var/run/mysqld/mysqld.pid

server-id = "1"
[root@mysqlserver ~]# systemctl restart mysqld
[root@mysqlserver ~]# 

mysql> show variables like 'server%';
+----------------+--------------------------------------+
| Variable_name  | Value                                |
+----------------+--------------------------------------+
| server_id      | 1                                    |
| server_id_bits | 32                                   |
| server_uuid    | 486f86fb-11db-11ef-9f50-000c29a50cb9 |
+----------------+--------------------------------------+
3 rows in set (0.00 sec)

In [ ]:
Change location of my.cnf from /etc to /etc/mysql
================================================================

[root@mysqlserver etc]# pwd
/etc
[root@mysqlserver etc]# mkdir mysql
[root@mysqlserver etc]# 
[root@mysqlserver etc]# cp -p my.cnf mysql/
[root@mysqlserver etc]# 
[root@mysqlserver etc]# mv my.cnf my.cnf.old
[root@mysqlserver etc]# 
[root@mysqlserver etc]# systemctl restart mysqld
[root@mysqlserver etc]# 

[root@mysqlserver etc]# mysql

Welcome to the MySQL monitor.  Commands end with ; or \g.
mysql> select @@hostname;
+-------------+
| @@hostname  |
+-------------+
| mysqlserver |
+-------------+
mysql> select @@server_id;
+-------------+
| @@server_id |
+-------------+
|           1 |
+-------------+

In [ ]:
Check which option file was read 
===================================================


start mysqld with strace and read the ouput
===================================================

[root@mysqlserver etc]# systemctl stop mysqld
[root@mysqlserver etc]# 

[root@mysqlserver etc]# strace mysqld

execve("/usr/sbin/mysqld", ["mysqld"], 0x7ffd492b3850 /* 24 vars */) = 0
:
stat("/etc/my.cnf", 0x7ffc6e78ee80)     = -1 ENOENT (No such file or directory)

stat("/etc/mysql/my.cnf", {st_mode=S_IFREG|0644, st_size=150, ...}) = 0
openat(AT_FDCWD, "/etc/mysql/my.cnf", O_RDONLY) = 3

read(3, "[mysqld]\n\ndatadir=/var/lib/mysql"..., 4096) = 150
read(3, "", 4096)                       = 0
close(3)                                = 0

stat("/usr/etc/my.cnf", 0x7ffc6e78ee80) = -1 ENOENT (No such file or directory)
stat("/root/.my.cnf", 0x7ffc6e78ee80)   = -1 ENOENT (No such file or directory)

In [ ]:
Use includedir to specify multiple option files
=========================================================

create 2 .cnf files as below 
==================================

[root@mysqlserver mysql-files]# pwd
/var/lib/mysql-files

[root@mysqlserver mysql-files]# cat mysqld_memory.cnf 
[mysqld]
innodb_buffer_pool_size = 268435456
innodb_buffer_pool_chunk_size = 268435456
[root@mysqlserver mysql-files]# 

[root@mysqlserver mysql-files]# cat mysqld_connections.cnf 
[mysqld]
max_connections = 300
max_user_connections = 200
mysqlx_max_connections = 100
[root@mysqlserver mysql-files]# 


include this directory in /et/mysql/my.cnf and restart 
===============================================================

[root@mysqlserver mysql-files]# cat /etc/mysql/my.cnf 
!includedir /var/lib/mysql-files/

[mysqld]

datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock

log-error=/var/log/mysqld.log
pid-file=/var/run/mysqld/mysqld.pid

server-id = "1"
[root@mysqlserver mysql-files]# 


[root@mysqlserver mysql-files]# systemctl restart mysqld
[root@mysqlserver mysql-files]# 


check the newly configured options
===============================================

mysql> show variables like '%innodb_buffer%';
+-------------------------------------+----------------+
| Variable_name                       | Value          |
+-------------------------------------+----------------+
| innodb_buffer_pool_chunk_size       | 268435456      |
| innodb_buffer_pool_size             | 268435456      |
+-------------------------------------+----------------+

mysql> show variables like '%connections%';
+------------------------+-------+
| Variable_name          | Value |
+------------------------+-------+
| max_connections        | 300   |
| max_user_connections   | 200   |
| mysqlx_max_connections | 100   |
+------------------------+-------+

In [ ]:
mysql Data Directory 
=================================

-- is the directory where all the data is stored.
-- default location is /var/lib/mysql.
-- during mysql installation mysql user is created with its home directory as /var/lib/mysql.
-- is controlled by datadir option/variable.

mysql> show variables like '%datadir%';
+---------------+-----------------+
| Variable_name | Value           |
+---------------+-----------------+
| datadir       | /var/lib/mysql/ |
+---------------+-----------------+
1 row in set (0.00 sec)


Change data directory to /var/lib/mysql/data 
========================================================

[root@mysqlserver mysql]# pwd
/var/lib/mysql
[root@mysqlserver mysql]# mkdir data
[root@mysqlserver mysql]# chown -R mysql:mysql data
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# vi /etc/mysql/my.cnf 
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# cat /etc/mysql/my.cnf | grep datadir
datadir=/var/lib/mysql/data
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# mv * data/
mv: cannot move 'data' to a subdirectory of itself, 'data/data'
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# ls -rtl
total 4
drwxr-xr-x. 9 mysql mysql 4096 May 19 17:18 data
[root@mysqlserver mysql]# 

[root@mysqlserver mysql]# systemctl start mysqld
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# mysql

Welcome to the MySQL monitor.  Commands end with ; or \g.

mysql> show variables like '%datadir%';
+---------------+----------------------+
| Variable_name | Value                |
+---------------+----------------------+
| datadir       | /var/lib/mysql/data/ |
+---------------+----------------------+
1 row in set (0.00 sec)

In [ ]:
MySQL Binary Logs 
================================

-- contains records of changes made to database known as events.
-- will be in binary encrypted format.
-- mysqlbinlog utility can be used to convert this into clear text.
-- very important for replication and point-in-time recovery.
-- default size is 1 gb , controlled by max_bin_log_size.
-- retention is controlled by binlog_expire_logs_seconds.

mysql> show variables like '%log_bin%';
+---------------------------------+----------------------------------+
| Variable_name                   | Value                            |
+---------------------------------+----------------------------------+
| log_bin                         | ON                               |
| log_bin_basename                | /var/lib/mysql/data/binlog       |
| log_bin_index                   | /var/lib/mysql/data/binlog.index |

mysql> show variables like '%binlog%';
+------------------------------------------------+----------------------+
| Variable_name                                  | Value                |
+------------------------------------------------+----------------------+
| binlog_expire_logs_auto_purge                  | ON                   |
| binlog_expire_logs_seconds                     | 2592000              |
| binlog_format                                  | ROW                  |

2592000/86400 = 30 days.

mysql> show binary logs;
+---------------+-----------+-----------+
| Log_name      | File_size | Encrypted |
+---------------+-----------+-----------+
| binlog.000001 |      1387 | No        |
| binlog.000002 |       180 | No        |
| binlog.000003 |       157 | No        |
| binlog.000004 |       335 | No        |
| binlog.000005 |      3087 | No        |
| binlog.000006 |       180 | No        |
| binlog.000007 |      2631 | No        |
| binlog.000008 |       157 | No        |
| binlog.000009 |  69407143 | No        |


mysql> show binlog events in 'binlog.000011';

+---------------+------+----------------+-----------+-------------+--------------------------------------------------------------------------------------------------------------------+
| Log_name      | Pos  | Event_type     | Server_id | End_log_pos | Info                                                                                                               |
+---------------+------+----------------+-----------+-------------+--------------------------------------------------------------------------------------------------------------------+
| binlog.000011 |    4 | Format_desc    |         1 |         126 | Server ver: 8.0.30, Binlog ver: 4                                                                                  |
| binlog.000011 |  126 | Previous_gtids |         1 |         157 |                                                                                                                    |
| binlog.000011 |  157 | Anonymous_Gtid |         1 |         234 | SET @@SESSION.GTID_NEXT= 'ANONYMOUS'                                                                               |
| binlog.000011 |  234 | Query          |         1 |         350 | drop database ordersdb /* xid=4 */                                                                                 |
| binlog.000011 |  350 | Anonymous_Gtid |         1 |         427 | SET @@SESSION.GTID_NEXT= 'ANONYMOUS'                                                                               |
| binlog.000011 |  427 | Query          |         1 |         538 | create database posdb /* xid=7 */                                                                                  |
| binlog.000011 |  538 | Anonymous_Gtid |         1 |         617 | SET @@SESSION.GTID_NEXT= 'ANONYMOUS'                                                                               |
| binlog.000011 |  617 | Query          |         1 |         799 | use `posdb`; create table employee_salaries (
emp_id int primary key,
emp_salary numeric(6,2)
) engine = blackhole |
| binlog.000011 |  799 | Anonymous_Gtid |         1 |         878 | SET @@SESSION.GTID_NEXT= 'ANONYMOUS'                                                                               |



Disable binary logging
=======================================

[root@mysqlserver mysql]# vi /etc/mysql/my.cnf
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# cat /etc/mysql/my.cnf | grep log-bin
disable-log-bin
[root@mysqlserver mysql]# systemctl restart mysqld
[root@mysqlserver mysql]# 

mysql> show variables like '%log_bin%';
+---------------------------------+-------+
| Variable_name                   | Value |
+---------------------------------+-------+
| log_bin                         | OFF   |
| log_bin_basename                |       |
| log_bin_index                   |       |

mysql> show binary logs;
ERROR 1381 (HY000): You are not using binary logging
mysql> 


Enable binary logging with different location
=====================================================

[root@mysqlserver mysql]# pwd
/var/lib/mysql
[root@mysqlserver mysql]# mkdir binlog
[root@mysqlserver mysql]# chown -R mysql:mysql binlog
[root@mysqlserver mysql]# 


[root@mysqlserver binlog]# vi /etc/mysql/my.cnf 
[root@mysqlserver binlog]# 
[root@mysqlserver binlog]# cat /etc/mysql/my.cnf | grep log_bin
log_bin=/var/lib/mysql/binlog/posb_binlog
log_bin_index=/var/lib/mysql/binlog/posb_binlog.index
[root@mysqlserver binlog]# 
[root@mysqlserver binlog]# systemctl restart mysqld
[root@mysqlserver binlog]# 

mysql> show variables like '%log_bin%';
+---------------------------------+-----------------------------------------+
| Variable_name                   | Value                                   |
+---------------------------------+-----------------------------------------+
| log_bin                         | ON                                      |
| log_bin_basename                | /var/lib/mysql/binlog/posb_binlog       |
| log_bin_index                   | /var/lib/mysql/binlog/posb_binlog.index |

mysql> show binary logs;
+--------------------+-----------+-----------+
| Log_name           | File_size | Encrypted |
+--------------------+-----------+-----------+
| posb_binlog.000001 |       180 | No        |
| posb_binlog.000002 |       180 | No        |
| posb_binlog.000003 |       157 | No        |
+--------------------+-----------+-----------+
3 rows in set (0.00 sec)

[root@mysqlserver binlog]# pwd
/var/lib/mysql/binlog
[root@mysqlserver binlog]# ls -rtl
total 16
-rw-r-----. 1 mysql mysql 180 May 19 18:17 posb_binlog.000001
-rw-r-----. 1 mysql mysql 180 May 19 20:22 posb_binlog.000002
-rw-r-----. 1 mysql mysql 123 May 19 20:22 posb_binlog.index
-rw-r-----. 1 mysql mysql 157 May 19 20:22 posb_binlog.000003
[root@mysqlserver binlog]# 


In [ ]:
MySQL Error Log file 
=============================================

-- contains records of startup and shutdown time.
-- contains diagnostic messages like errors , warnings.
-- default location is /var/log/mysqld.log
-- controlled by log-error system variable.

mysql> show variables like '%log_error%';
+----------------------------+----------------------------------------+
| Variable_name              | Value                                  |
+----------------------------+----------------------------------------+
| log_error                  | /var/log/mysqld.log    


Change the location of error log file 
============================================

[root@mysqlserver mysql]# pwd
/var/lib/mysql
[root@mysqlserver mysql]# mkdir errorlog
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# chown -R mysql:mysql errorlog/
[root@mysqlserver mysql]# 

[root@mysqlserver errorlog]# vi /etc/mysql/my.cnf 
[root@mysqlserver errorlog]# cat /etc/mysql/my.cnf | grep log_error
log_error = /var/lib/mysql/errorlog/posb_mysqld.log
[root@mysqlserver errorlog]# 
[root@mysqlserver errorlog]# systemctl restart mysqld
[root@mysqlserver errorlog]# 

mysql> show variables like '%log_error%';
+----------------------------+-----------------------------------------+
| Variable_name              | Value                                   |
+----------------------------+-----------------------------------------+
| log_error                  | /var/lib/mysql/errorlog/posb_mysqld.log |

mysql> \! ls -rtl /var/lib/mysql/errorlog/posb_mysqld.log

-rw-r-----. 1 mysql mysql 1122 May 19 20:38 /var/lib/mysql/errorlog/posb_mysqld.log

In [ ]:
MySQL Temp Directory
===============================

-- is the directory where mysql stores temporary files.
-- default location is /tmp and is controlled by tmpdir system variable.

mysql> show variables like '%tmpdir%';
+---------------------+-------+
| Variable_name       | Value |
+---------------------+-------+
| innodb_tmpdir       |       |
| replica_load_tmpdir | /tmp  |
| slave_load_tmpdir   | /tmp  |
| tmpdir              | /tmp  |
+---------------------+-------+


Change the temp directory location 
============================================

[root@mysqlserver mysql]# pwd
/var/lib/mysql
[root@mysqlserver mysql]# mkdir tmpdir
[root@mysqlserver mysql]# 
[root@mysqlserver mysql]# chown -R mysql:mysql tmpdir/
[root@mysqlserver mysql]# 

[root@mysqlserver tmpdir]# vi /etc/mysql/my.cnf 
[root@mysqlserver tmpdir]# 
[root@mysqlserver tmpdir]# cat /etc/mysql/my.cnf | grep tmpdir
tmpdir = /var/lib/mysql/tmpdir
[root@mysqlserver tmpdir]# 

[root@mysqlserver tmpdir]# systemctl restart mysqld
[root@mysqlserver tmpdir]# 
[root@mysqlserver tmpdir]# mysql
Welcome to the MySQL monitor.  Commands end with ; or \g.

mysql> show variables like '%tmpdir%';
+---------------------+-----------------------+
| Variable_name       | Value                 |
+---------------------+-----------------------+
| innodb_tmpdir       |                       |
| replica_load_tmpdir | /var/lib/mysql/tmpdir |
| slave_load_tmpdir   | /var/lib/mysql/tmpdir |
| tmpdir              | /var/lib/mysql/tmpdir |
+---------------------+-----------------------+
4 rows in set (0.01 sec)